In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Sine(nn.Module):
    """This class defines the Sine activation function as a nn.Module"""
    def __init__(self):
        super(Sine, self).__init__()

    def forward(self, x):
        return torch.sin(x)
    
    
class LSTM(nn.Module):

    def __init__(self, layers, stable, activation):
        super(LSTM, self).__init__()

        self.Wi = nn.Linear(in_features=layers[0], out_features=layers[1])

        self.Wf = nn.Linear(in_features=layers[0], out_features=layers[1])

        self.Wg = nn.Linear(in_features=layers[0], out_features=layers[1])

        self.Wo = nn.Linear(in_features=layers[0], out_features=layers[1])
        
        self.Wout = nn.Linear(in_features=layers[1], out_features=layers[2])
        
        self.activation = activation

        self.epsilon = 0.01
        self.stable = stable

    def forward(self, x):
        
        i = nn.Sigmoid()(self.Wi(x))
        f = nn.Sigmoid()(self.Wf(x))
        g = nn.Tanh()(self.Wg(x))
        o = nn.Sigmoid()(self.Wo(x))
        c_new = i*g
        h_new = o*nn.Tanh()(c_new)
        out = self.Wout(h_new)
        out = self.activation(out)
        return out


class Resnet(nn.Module):

    def __init__(self, layers, stable, activation):
        super(Resnet, self).__init__()

        self.layer1 = nn.Linear(in_features=layers[0], out_features=layers[1])
        self.layer2 = nn.Linear(in_features=layers[1], out_features=layers[2])
        self.layer2_input = nn.Linear(in_features=layers[0], out_features=layers[2])
        self.layer3 = nn.Linear(in_features=layers[2], out_features=layers[3])
        self.layer3_input = nn.Linear(in_features=layers[0], out_features=layers[3])
        self.layer4 = nn.Linear(in_features=layers[3], out_features=layers[4])
        self.layer4_input = nn.Linear(in_features=layers[0], out_features=layers[4])
        self.layer5 = nn.Linear(in_features=layers[4], out_features=layers[5])

        self.activation = activation

        self.epsilon = 0.01
        self.stable = stable

    def stable_forward(self, layer, out):  # Building block for the NAIS-Net
        weights = layer.weight
        delta = 1 - 2 * self.epsilon
        RtR = torch.matmul(weights.t(), weights)
        norm = torch.norm(RtR)
        if norm > delta:
            RtR = delta ** (1 / 2) * RtR / (norm ** (1 / 2))
        A = RtR + torch.eye(RtR.shape[0]).cuda() * self.epsilon

        return F.linear(out, -A, layer.bias)

    def forward(self, x):
        u = x

        out = self.layer1(x)
        out = self.activation(out)

        shortcut = out
        if self.stable:
            out = self.stable_forward(self.layer2, out)
            out += self.layer2_input(u)
        else:
            out = self.layer2(out)
        out = self.activation(out)
        out += shortcut

        shortcut = out
        if self.stable:
            out = self.stable_forward(self.layer3, out)
            out += self.layer3_input(u)
        else:
            out = self.layer3(out)
        out = self.activation(out)
        out += shortcut

        shortcut = out
        if self.stable:
            out = self.stable_forward(self.layer4, out)
            out += self.layer4_input(u)
        else:
            out = self.layer4(out)

        out = self.activation(out)
        out += shortcut

        out = self.layer5(out)

        return out


class SDEnet(nn.Module):

    def __init__(self, layers, activation):
        super(SDEnet, self).__init__()

        self.layers = nn.ModuleList()
        self.brownian = nn.ModuleList()

        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(in_features=layers[i], out_features=layers[i + 1]))
            if i > 0 and i < len(layers) - 2:
                self.brownian.append(nn.Linear(in_features=layers[i], out_features=1, bias=False))

        self.activation = activation
        self.epsilon = 1e-4
        self.h = 0.1

    def product(self, layer, out):
        weights = layer.weight
        RtR = torch.matmul(weights.t(), weights)
        A = RtR + torch.eye(RtR.shape[0]).cuda() * self.epsilon

        return F.linear(out, A, layer.bias)

    def forward(self, x):
        out = self.layers[0](x)
        out = self.activation(out)

        for i, layer in enumerate(self.layers[1:-1]):
            shortcut = out
            out = layer(out)
            out = shortcut + self.h * self.activation(out) + self.h ** (1 / 2) * self.product(self.brownian[i],
                                                                                              torch.rand_like(out))
            # out = shortcut + self.activation(out) + 0.4*torch.ones_like(out)*torch.rand_like(out)

        out = self.layers[-1](out)

        return out


class VerletNet(nn.Module):

    def __init__(self, layers, activation):
        super(VerletNet, self).__init__()

        self.layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(in_features=layers[i], out_features=layers[i + 1]))

        self.h = 0.5
        self.activation = activation

    def transpose(self, layer, out):

        return F.linear(out, layer.weight.t(), layer.bias)

    def forward(self, x):

        out = self.layers[0](x)
        out = self.activation(out)

        z = torch.zeros_like(out)

        for layer in self.layers[1:-1]:
            shortcut = out
            out = self.transpose(layer, out)
            z = z - self.activation(out)
            out = layer(z)
            out = shortcut + self.activation(out)

        out = self.layers[-1](out)

        return out


class VerletNet(nn.Module):

    def __init__(self, layers, activation):
        super(VerletNet, self).__init__()

        self.layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(in_features=layers[i], out_features=layers[i + 1]))

        self.h = 0.5
        self.activation = activation

    def transpose(self, layer, out):
        return F.linear(out, layer.weight.t(), layer.bias)

    def forward(self, x):

        out = self.layers[0](x)
        out = self.activation(out)

        z = torch.zeros_like(out)

        for layer in self.layers[1:-1]:
            shortcut = out
            out = self.transpose(layer, out)
            z = z - self.activation(out)
            out = layer(z)
            out = shortcut + self.activation(out)

        out = self.layers[-1](out)

        return out

In [ ]:
import numpy as np
from abc import ABC, abstractmethod
import time

import torch
import torch.nn as nn
import torch.optim as optim

#from Models import Resnet, Sine


class FBSNN(ABC):
    def __init__(self, Xi, T, M, N, D, layers, mode, activation):
        device_idx = 0
        if torch.cuda.is_available():
            self.device = torch.device("cuda:" + str(device_idx) if torch.cuda.is_available() else "cpu")
            torch.backends.cudnn.deterministic = True
        else:
            self.device = torch.device("cpu")

        #  We set a random seed to ensure that your results are reproducible
        # torch.manual_seed(0)

        self.Xi = torch.from_numpy(Xi).float().to(self.device)  # initial point
        self.Xi.requires_grad = True

        self.T = T  # terminal time
        self.M = M  # number of trajectories
        self.N = N  # number of time snapshots
        self.D = D  # number of dimensions
        self.mode = mode  # architecture: FC, Resnet and NAIS-Net are available
        self.activation = activation
        if activation == "Sine":
            self.activation_function = Sine()
        elif activation == "ReLU":
            self.activation_function = nn.ReLU()

        # initialize NN
        if self.mode == "FC":
            self.layers = []
            for i in range(len(layers) - 2):
                self.layers.append(nn.Linear(in_features=layers[i], out_features=layers[i + 1]))
                self.layers.append(self.activation_function)
            self.layers.append(nn.Linear(in_features=layers[-2], out_features=layers[-1]))

            self.model = nn.Sequential(*self.layers).to(self.device)

        elif self.mode == "NAIS-Net":
            self.model = Resnet(layers, stable=True, activation=self.activation_function).to(self.device)
        elif self.mode == "Resnet":
            self.model = Resnet(layers, stable=False, activation=self.activation_function).to(self.device)
        elif self.mode == "Verlet":
            self.model = VerletNet(layers, activation=self.activation_function).to(self.device)
        elif self.mode == "SDEnet":
            self.model = SDEnet(layers, activation=self.activation_function).to(self.device)
        elif self.mode == "LSTM":
            self.model = LSTM(layers, stable=True, activation=self.activation_function).to(self.device)

        self.model.apply(self.weights_init)
        #print(self.model)

        # Record the loss

        self.training_loss = []
        self.iteration = []

    def weights_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)

    #calculates next Yt and Zt value
    # also calculates initail Y0 and Z0 calue
    def net_u(self, t, X):  # M x 1, M x D

        input = torch.cat((t, X), 1)
        u = self.model(input)  # M x 1
        Du = torch.autograd.grad(outputs=[u], inputs=[X], grad_outputs=torch.ones_like(u), allow_unused=True,
                                 retain_graph=True, create_graph=True)[0]
        return u, Du

    def Dg_tf(self, X):  # M x D

        g = self.g_tf(X)
        Dg = torch.autograd.grad(outputs=[g], inputs=[X], grad_outputs=torch.ones_like(g), allow_unused=True,
                                 retain_graph=True, create_graph=True)[0]  # M x D
        return Dg

    def loss_function(self, t, W, Xi):
        loss = 0
        X_list = []
        Y_list = []

        t0 = t[:, 0, :]
        W0 = W[:, 0, :]

        X0 = Xi.repeat(self.M, 1).view(self.M, self.D)  # M x D
        #there are M separate samples of initial inputs
        Y0, Z0 = self.net_u(t0, X0)  # M x 1, M x D
        #M samples of outputs and change in outputs (Z)

        X_list.append(X0)
        Y_list.append(Y0)

        size = t.shape[1] #size = number of timestamps N

        for n in range(0, size-1): #iterates through the timestamps
            # print(n)
            # print(size)
            # print(t)
            #specifies the current timestamp
            t1 = t[:, n + 1, :] 
            W1 = W[:, n + 1, :]
            #calculating each time step in equation (7)
            X1 = X0 + self.mu_tf(t0, X0, Y0, Z0) * (t1 - t0) + torch.squeeze(
                torch.matmul(self.sigma_tf(t0, X0, Y0), (W1 - W0).unsqueeze(-1)), dim=-1)
            Y1_tilde = Y0 + self.phi_tf(t0, X0, Y0, Z0) * (t1 - t0) + torch.sum(
                Z0 * torch.squeeze(torch.matmul(self.sigma_tf(t0, X0, Y0), (W1 - W0).unsqueeze(-1))), dim=1,
                keepdim=True)
            #this gets the next values of Y1 based on the NN model
            Y1, Z1 = self.net_u(t1, X1) 

            loss += torch.sum(torch.pow(Y1 - Y1_tilde, 2))
            #Y1_tilde = Y^n_m + phi*delta_t + (Z^n_m)'*Sigma^n_m*deltaW  

            t0 = t1
            W0 = W1
            X0 = X1 # M x D
            Y0 = Y1 # M x 1
            Z0 = Z1

            X_list.append(X0)
            Y_list.append(Y0)

        #Y1 is the "payoff" for final time
        #self.g_tf
        loss += torch.sum(torch.pow(Y1 - self.g_tf(X1), 2))
        loss += torch.sum(torch.pow(Z1 - self.Dg_tf(X1), 2)) # can remove this and perform training too
        #now we have iterated over all the timestamps and have calcualted the losses and accumulated the X's and Y's used to caluclate the losses

        #length of X_list and Y_list is N + 1 (number of timestamps + 1)
        # + 1 is present due to beginning and ending times [0,T]
        # ie: for N = 2 -> time steps are [0, T/2, T]
        X = torch.stack(X_list, dim=1) #shape: M x N+1 x D
        Y = torch.stack(Y_list, dim=1) #shape: M x N+1 x 1

        #returns loss (scalar), X (vector of all the X's generated), Y (vector of all resulting Y's), Y0
        return loss, X, Y, Y[0, 0, 0]

    def fetch_minibatch_train(self, size):  # Generate time + a Brownian motion
        T = self.T

        M = self.M #number of samples that we are going to generate
        N = size #number of timesteps that will be uised
        D = self.D #number of dimensions

        Dt = np.zeros((M, N + 1, 1))  # M x (N+1) x 1
        DW = np.zeros((M, N + 1, D))  # M x (N+1) x D

        dt = T / N

        Dt[:, 1:, :] = dt
        DW[:, 1:, :] = np.sqrt(dt) * np.random.normal(size=(M, N, D))

        #cumsum = cumulative sum
        t = np.cumsum(Dt, axis=1)  # M x (N+1) x 1
        W = np.cumsum(DW, axis=1)  # M x (N+1) x D
        t = torch.from_numpy(t).float().to(self.device)
        W = torch.from_numpy(W).float().to(self.device)

        return t, W

    def fetch_minibatch(self):  # Generate time + a Brownian motion
        T = self.T

        M = self.M
        N = self.N
        D = self.D

        Dt = np.zeros((M, N + 1, 1))  # M x (N+1) x 1
        DW = np.zeros((M, N + 1, D))  # M x (N+1) x D

        dt = T / N

        Dt[:, 1:, :] = dt
        DW[:, 1:, :] = np.sqrt(dt) * np.random.normal(size=(M, N, D))

        t = np.cumsum(Dt, axis=1)  # M x (N+1) x 1
        W = np.cumsum(DW, axis=1)  # M x (N+1) x D
        t = torch.from_numpy(t).float().to(self.device)
        W = torch.from_numpy(W).float().to(self.device)

        return t, W

    def train(self, N_Iter, learning_rate, L, h_Factor, fixed=0):
        '''
          L = number of layers that will be performed
          h_Factor = factor in which each number of timesteps will be divided against
        '''
        loss_temp = np.array([])

        previous_it = 0
        if self.iteration != []:
            previous_it = self.iteration[-1]
          
        #want to divide evently across the layers
        layerIters = N_Iter // L

        # Optimizers
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

        start_time = time.time()
        time_per_epoch = []
        
        numLayers = []
        for it in range(previous_it, previous_it + N_Iter):
            #size = number of time steps between [0,T]
            #this is the point where multilayer monte carlo is used------

            if fixed==0:
              l = it//layerIters+1 #layers 1 to L
              size = h_Factor**l       
              if numLayers == [] or l != numLayers[-1][0]:
                numLayers.append([l,size])   
            else:
              size = fixed



            # if it < N_Iter/5:
            #   size = 2
            # elif N_Iter/5 <= it < (2*N_Iter)/5:
            #   size = 4
            # elif (2*N_Iter)/5 <= it < (3*N_Iter)/5:
            #   size = 8
            # elif (3*N_Iter)/5 <= it < (4*N_Iter)/5:
            #   size = 16
            # else:
            #   size = 32
            #------------------------------------------------------------

            self.optimizer.zero_grad()

            #this step fetches the samples of the number of timestamps that will be used
            t_batch, W_batch = self.fetch_minibatch_train(size)  # M x (N+1) x 1, M x (N+1) x D
            #---------------------------------------------------------------------------
            '''
            The vectors are as:
            [M, N+1, 1] or [M, N+1, D]
            -> index 0: number of samples that will be run
            -> index 1: number of timesteps run for each sample
            -> index 2: number of dimensions
              -> time only has 1 dimensions
              -> Brownian motion has 5 dimensions for 5 dimensional input

            '''

            loss, X_pred, Y_pred, Y0_pred = self.loss_function(t_batch, W_batch, self.Xi)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            loss_temp = np.append(loss_temp, loss.cpu().detach().numpy())

            # Print
            
            if it % 100 == 0:
                elapsed = time.time() - start_time
                time_per_epoch.append(elapsed)
                print('It: %d, Loss: %.3e, Y0: %.3f, Time: %.2f, Learning Rate: %.3e' %
                      (it, loss, Y0_pred, elapsed, learning_rate))
                start_time = time.time()


            # Loss
            if it % 100 == 0:
                self.training_loss.append(loss_temp.mean())
                loss_temp = np.array([])

                self.iteration.append(it)

            graph = np.stack((self.iteration, self.training_loss))
        plt.plot(list(np.linspace(0,N_Iter-100,int(N_Iter/100))), time_per_epoch)
        plt.xlabel("epoch")
        plt.ylabel("time(seconds)")
        plt.title("Time in seconds per epoch")
        print("Number of layers arr", numLayers)
        return graph

    def predict(self, Xi_star, t_star, W_star):
        Xi_star = torch.from_numpy(Xi_star).float().to(self.device)
        Xi_star.requires_grad = True
        loss, X_star, Y_star, Y0_pred = self.loss_function(t_star, W_star, Xi_star)

        return X_star, Y_star

    ###########################################################################
    ############################# Change Here! ################################
    ###########################################################################
    @abstractmethod
    def phi_tf(self, t, X, Y, Z):  # M x 1, M x D, M x 1, M x D
        pass  # M x1

    @abstractmethod
    def g_tf(self, X):  # M x D
        pass  # M x 1

    @abstractmethod
    def mu_tf(self, t, X, Y, Z):  # M x 1, M x D, M x 1, M x D
        M = self.M
        D = self.D
        return torch.zeros([M, D]).to(self.device)  # M x D

    @abstractmethod
    def sigma_tf(self, t, X, Y):  # M x 1, M x D, M x 1
        M = self.M
        D = self.D
        return torch.diag_embed(torch.ones([M, D])).to(self.device)  # M x D x D
    ###########################################################################



In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

#from FBSNNs import FBSNN

"""
t1 = t[:, n + 1, :]
W1 = W[:, n + 1, :]
#calculating each time step in equation (7)
X1 = X0 + self.mu_tf(t0, X0, Y0, Z0) * (t1 - t0) + torch.squeeze(
    torch.matmul(self.sigma_tf(t0, X0, Y0), (W1 - W0).unsqueeze(-1)), dim=-1)
Y1_tilde = Y0 + self.phi_tf(t0, X0, Y0, Z0) * (t1 - t0) + torch.sum(
    Z0 * torch.squeeze(torch.matmul(self.sigma_tf(t0, X0, Y0), (W1 - W0).unsqueeze(-1))), dim=1,
    keepdim=True)
Y1, Z1 = self.net_u(t1, X1)

"""

class BlackScholesBarenblatt(FBSNN):
    def __init__(self, Xi, T, M, N, D, layers, mode, activation):

        super().__init__(Xi, T, M, N, D, layers, mode, activation)

    #used to calculate Y_{n+1}
    def phi_tf(self, t, X, Y, Z):  # M x 1, M x D, M x 1, M x D
        interestRate = 0.05
        return interestRate * (Y - torch.sum(X * Z, dim=1, keepdim=True))  # M x 1

    #used to calculate the terminal value
    def g_tf(self, X):  # M x D
        return torch.sum(X ** 2, 1, keepdim=True)  # M x 1 
        # g(s) = |x|^2

    #for BSB eqn, mu_tf = 0
    def mu_tf(self, t, X, Y, Z):  # M x 1, M x D, M x 1, M x D
        return super().mu_tf(t, X, Y, Z)  # M x D

    #used to calculate X_{n+1} and Y_{n+1}
    def sigma_tf(self, t, X, Y):  # M x 1, M x D, M x 1
        sigma = 0.4
        return sigma * torch.diag_embed(X)  # M x D x D

    ###########################################################################


#equation (16), used to calculate loss
# only used for to generate test graph (plotting 100-dimensional B-S-B eq) where comparing learned with exact
def u_exact(t, X):  # (N+1) x 1, (N+1) x D
    r = 0.05
    sigma_max = 0.4
    return np.exp((r + sigma_max ** 2) * (T - t)) * np.sum(X ** 2, 1, keepdims=True)  # (N+1) x 1


def run_model(model, N_Iter, learning_rate, L, h_Factor, fixed=0):
    tot = time.time()
    samples = 5
    print(model.device)
    graph = model.train(N_Iter, learning_rate, L, h_Factor, fixed=fixed)
    print("total time:", time.time() - tot, "s")

     
    np.random.seed(42)
    t_test, W_test = model.fetch_minibatch()
    X_pred, Y_pred = model.predict(Xi, t_test, W_test)


    if type(t_test).__module__ != 'numpy':
        t_test = t_test.cpu().numpy()
    if type(X_pred).__module__ != 'numpy':
        X_pred = X_pred.cpu().detach().numpy()
    if type(Y_pred).__module__ != 'numpy':
        Y_pred = Y_pred.cpu().detach().numpy()

    Y_test = np.reshape(u_exact(np.reshape(t_test[0:M, :, :], [-1, 1]), np.reshape(X_pred[0:M, :, :], [-1, D])),
                        [M, -1, 1])

    plt.figure()
    plt.plot(graph[0], graph[1])
    plt.xlabel('Iterations')
    plt.ylabel('Value')
    plt.yscale("log")
    plt.title('Evolution of the training loss')
    # plt.title('Evolution of the training loss')
    plt.savefig(str(D) + '-dimensional Black-Scholes-Barenblatt loss, ' + model.mode + "-" + model.activation +"-iters:"+str(N_Iter)+"-L:"+str(L)+",hFactor:"+str(h_Factor) + ".pdf", bbox_inches='tight')



    plt.figure()
    plt.plot(t_test[0:1, :, 0].T, Y_pred[0:1, :, 0].T, 'b', label='Learned $u(t,X_t)$')
    plt.plot(t_test[0:1, :, 0].T, Y_test[0:1, :, 0].T, 'r--', label='Exact $u(t,X_t)$')
    plt.plot(t_test[0:1, -1, 0], Y_test[0:1, -1, 0], 'ko', label='$Y_T = u(T,X_T)$')

    plt.plot(t_test[1:samples, :, 0].T, Y_pred[1:samples, :, 0].T, 'b')
    plt.plot(t_test[1:samples, :, 0].T, Y_test[1:samples, :, 0].T, 'r--')
    plt.plot(t_test[1:samples, -1, 0], Y_test[1:samples, -1, 0], 'ko')

    plt.plot([0], Y_test[0, 0, 0], 'ks', label='$Y_0 = u(0,X_0)$')

    plt.xlabel('$t$')
    plt.ylabel('$Y_t = u(t,X_t)$')
    plt.title(str(D) + '-dimensional Black-Scholes-Barenblatt paths, ' + model.mode + "-" + model.activation+"-iters:"+str(N_Iter)+"-L:"+str(L)+",hFactor:"+str(h_Factor))
    # plt.legend()
    plt.legend(bbox_to_anchor=(1.02, 1.02))
    plt.savefig(str(D) + '-dimensional Black-Scholes-Barenblatt paths, ' + model.mode + "-" + model.activation +"-iters:"+str(N_Iter)+"-L:"+str(L)+",hFactor:"+str(h_Factor) + ".pdf", bbox_inches='tight')

    errors = np.sqrt((Y_test - Y_pred) ** 2 / Y_test ** 2)
    mean_errors = np.mean(errors, 0)
    std_errors = np.std(errors, 0)

    plt.figure()
    plt.plot(t_test[0, :, 0], mean_errors, 'b', label='mean')
    plt.plot(t_test[0, :, 0], mean_errors + 2 * std_errors, 'r--', label='mean + two standard deviations')
    plt.xlabel('$t$')
    plt.ylabel('relative error')
    plt.title(str(D) + '-dimensional Black-Scholes-Barenblatt, ' + model.mode + "-" + model.activation +"-iters:"+str(N_Iter)+"-L:"+str(L)+",hFactor:"+str(h_Factor))
    # plt.legend()
    # plt.savefig(str(D) + '-dimensional Black-Scholes-Barenblatt, ' + model.mode + "-" + model.activation)
    plt.legend(bbox_to_anchor=(1.02, 1.02))
    plt.savefig(str(D) + '-dimensional Black-Scholes-Barenblatt, ' + model.mode + "-" + model.activation +"-iters:"+str(N_Iter)+"-L:"+str(L)+",hFactor:"+str(h_Factor)+ ".pdf", bbox_inches='tight')


if __name__ == "__main__":
    tot = time.time()
    M = 10  # number of trajectories (batch size)
      #in the paper, M = 100
    N = 50  # number of time snapshots
    D = 10  # number of dimensions

    layers = [D + 1] + 4 * [256] + [1] #represents the number of neurons for each layer
    #there are 4 hidden layers of 256 neurons
    #layers = [D + 1] + [256] +[1]

    Xi = np.array([1.0, 0.5] * int(D / 2))[None, :] #initaliztion of X0, sahpe (1,10)
    T = 1.0 #total time
    "Available architectures"

In [ ]:
    mode = "FC"
    activation = "ReLU"  # Sine and ReLU are available
    learning_rate = 1e-3
    iterations = 2*10**3
    model = BlackScholesBarenblatt(Xi, T,
                                   M, N, D,
                                   layers, mode, activation)
    L = 5
    h_Factor = 2
    run_model(model, iterations, learning_rate, L, h_Factor)
    #the number of iterations and learning rate are the same as the Raissi Paper for the first go around

In [ ]:
mode = "FC"
activation = "ReLU"  # Sine and ReLU are available
learning_rate = 1e-3
iterations = 2*10**3
model = BlackScholesBarenblatt(Xi, T,
                                M, N, D,
                                layers, mode, activation)
L = 1 #are not used since fixed is set
h_factor=1 #are not used since fixed is set
run_model(model, iterations, learning_rate, L,h_factor,fixed=32)
#the number of iterations and learning rate are the same as the Raissi Paper for the first go around